<a href="https://colab.research.google.com/github/Annie00000/Project/blob/main/1_6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## ANOVA + 卡方檢定

要將 ANOVA（組間差異） 與 Cramer's V（類別相關性） 整合，最科學的方式是使用 「解釋變異量」 的概念。

**1. 指標構建：變異解釋力綜合法**

我們不直接使用 P-value（因為 P-value 隨樣本數波動大），而是使用 ANOVA 中的 Eta-squared ($\eta^2$)。它代表機台變量可以解釋 $y$ 欄位多少比例的變異。

*核心公式：*
    $$Score = \frac{W_1 \cdot \eta^2 + W_2 \cdot V}{W_1 + W_2}$$


- $\eta^2$ (Eta-squared)：來自 ANOVA。計算機台類別對 $y$ 值的貢獻度。
- $V$ (Cramer's V)：來自卡方檢定。計算機台類別與 Cluster 分群結果的吻合度。
- $W_1, W_2$：權重（建議 $0.5 : 0.5$，代表您同時看重「數值表現 $y$」與「群體分類」）。


**2. 各指標的標準化與計算方式**

> ANOVA 部分： $\eta^2$ (效果量)

不要只看 P-value，要算 $\eta^2$。

$$\eta^2 = \frac{SS_{between}}{SS_{total}}$$


  - 解釋：如果 $\eta^2 = 0.8$，代表這個機台的類別設定，解釋了 $y$ 欄位 $80\%$ 的變動。

  - P-value 輔助：如果 $P > 0.05$，您可以直接強迫該機台的 $\eta^2 = 0$，代表不具信度。


> 相關性部分：Cramer's V

這原本就是 $0 \sim 1$ 的指標。
- 解釋：如果 $V=1$，代表機台類別完全對應 Cluster (例如：A機台產出的全是 $g1$)。



In [ ]:
import pandas as pd
import numpy as np
from scipy import stats

def calculate_composite_score(df, machine_col, y_col, cluster_col):
    # 1. ANOVA: 計算 Eta-squared
    groups = [group[y_col].values for name, group in df.groupby(machine_col)]
    f_stat, p_anova = stats.f_oneway(*groups)

    # 計算 Eta-squared: SS_between / SS_total
    overall_mean = df[y_col].mean()
    ss_total = ((df[y_col] - overall_mean)**2).sum()
    ss_between = sum([len(g) * (g.mean() - overall_mean)**2 for g in groups])
    eta_sq = ss_between / ss_total if p_anova < 0.05 else 0 # 若不顯著則歸零

    # 2. Correlation: 計算 Cramer's V
    confusion_matrix = pd.crosstab(df[machine_col], df[cluster_col])
    chi2 = stats.chi2_contingency(confusion_matrix)[0]
    n = len(df)
    r, k = confusion_matrix.shape
    v = np.sqrt(chi2 / (n * min(k-1, r-1)))

    # 3. 綜合指標 (等權重)
    score = (eta_sq + v) / 2

    return {'Machine': machine_col, 'Eta_sq': eta_sq, 'Cramer_V': v, 'Final_Score': score}

# 執行分析
results = []
for col in ['EPI_1', 'EPI_2', 'MES_1']:
    results.append(calculate_composite_score(df, col, 'y', 'cluster'))

summary_df = pd.DataFrame(results).sort_values('Final_Score', ascending=False)